In [ ]:
!pip install auto-round
!pip install auto-gptq accelerate datasets optimum
!pip install -U bitsandbytes
!pip install autoawq 
!pip install --upgrade torch

In [9]:
# from huggingface_hub import login
# read_token = ""
# write_token = ''
# login(token = read_token)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

In [4]:
model_name = "meta-llama/Llama-3.2-1B"

In [ ]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('Size (KB):', os.path.getsize("temp_delme.p")/1e3)
    os.remove('temp_delme.p')

## AutoRound

In [17]:
from auto_round import AutoRound

In [ ]:
model_for_auto_round = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)
tokenizer_for_auto_round = AutoTokenizer.from_pretrained(model_name)

In [ ]:
bits, group_size, sym = 4, 128, True
autoround = AutoRound(model, tokenizer, bits=bits, group_size=group_size, batch_size=2, seqlen=512, sym=sym, gradient_accumulate_steps=4, device='cuda')
autoround.quantize()

autoround.save_quantized(model_name +"-auto-round-4bit")

In [ ]:
model_for_auto_round.push_to_hub(model_name.split('/')[1] + "-auto-round-4bit", token=write_token)
tokenizer_for_auto_round.push_to_hub(model_name.split('/')[1] + "-auto-round-4bit",token=write_token)

## AWQ

In [2]:
from awq import AutoAWQForCausalLM

In [5]:
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }
model_for_awq = AutoAWQForCausalLM.from_pretrained(model_name, safetensors=True, device_map='cuda')
tokenizer_for_awq = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [8]:
model_for_awq.quantize(tokenizer_for_awq, quant_config=quant_config)

# Save quantized model with safetensors
model_for_awq.save_quantized(model_name +"-awq-4-bit", safetensors=True)
tokenizer_for_awq.save_pretrained(model_name +"-awq-4-bit")

README.md:   0%|          | 0.00/167 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


val.jsonl.zst:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/214670 [00:00<?, ? examples/s]

AWQ: 100%|██████████| 16/16 [09:15<00:00, 34.70s/it]


('Llama-3.2-1B-awq-4-bit/tokenizer_config.json',
 'Llama-3.2-1B-awq-4-bit/special_tokens_map.json',
 'Llama-3.2-1B-awq-4-bit/tokenizer.json')

In [15]:
model_for_awq = AutoModelForCausalLM.from_pretrained(model_name + "-awq-4-bit", torch_dtype=torch.float16, device_map='auto')
tokenizer_for_awq = AutoTokenizer.from_pretrained(model_name + "-awq-4-bit")

In [19]:
model_for_awq.push_to_hub(model_name.split('/')[1] +"-awq-4-bit", token=write_token)
tokenizer_for_awq.push_to_hub(model_name.split('/')[1] +"-awq-4-bit",token=write_token)

model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/skshmjn/Llama-3.2-1B-awq-4-bit/commit/3e27b8ebdc9bcbf6cbf41e00fa4918e236fd141f', commit_message='Upload tokenizer', commit_description='', oid='3e27b8ebdc9bcbf6cbf41e00fa4918e236fd141f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/skshmjn/Llama-3.2-1B-awq-4-bit', endpoint='https://huggingface.co', repo_type='model', repo_id='skshmjn/Llama-3.2-1B-awq-4-bit'), pr_revision=None, pr_num=None)

## GPTQ

In [20]:
from optimum.gptq import GPTQQuantizer

/usr/local/lib/python3.10/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/usr/local/lib/python3.10/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/usr/local/lib/python3.10/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


In [21]:
model_for_gptq = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
tokenizer_for_gptq = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [23]:
quantizer = GPTQQuantizer(bits=4, dataset="c4", model_seqlen = 2048)
quantized_model = quantizer.quantize_model(model_for_gptq, tokenizer_for_gptq)

README.md:   0%|          | 0.00/41.1k [00:00<?, ?B/s]

c4-train.00000-of-01024.json.gz:   0%|          | 0.00/319M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Quantizing model.layers blocks :   0%|          | 0/16 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5055: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [24]:
quantized_model.save_pretrained(model_name + "-GPTQ-4bit", safetensors=True)
tokenizer_for_gptq.save_pretrained(model_name + "-GPTQ-4bit")

('meta-llama/Llama-3.2-1B-GPTQ-4bit/tokenizer_config.json',
 'meta-llama/Llama-3.2-1B-GPTQ-4bit/special_tokens_map.json',
 'meta-llama/Llama-3.2-1B-GPTQ-4bit/tokenizer.json')

In [26]:
quantized_model.push_to_hub(model_name.split('/')[1] + "-GPTQ-4bit", token=write_token)
tokenizer_for_gptq.push_to_hub(model_name.split('/')[1] + "-GPTQ-4bit",token=write_token)

model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/skshmjn/Llama-3.2-1B-GPTQ-4bit/commit/a654ac6030376465d6ad4fe54b65fa58ac1c1e75', commit_message='Upload tokenizer', commit_description='', oid='a654ac6030376465d6ad4fe54b65fa58ac1c1e75', pr_url=None, repo_url=RepoUrl('https://huggingface.co/skshmjn/Llama-3.2-1B-GPTQ-4bit', endpoint='https://huggingface.co', repo_type='model', repo_id='skshmjn/Llama-3.2-1B-GPTQ-4bit'), pr_revision=None, pr_num=None)